In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ["OPENAI_API_KEY"] 

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken


In [4]:
from langchain.text_splitter import CharacterTextSplitter

In [5]:
txt_file_path = "/home/lillian/Documents/TenAcademy/week11/Contract-AI-Chatbot/backend/data/contract.txt"

In [8]:
loader = TextLoader(file_path= txt_file_path, encoding="utf-8",)
data = loader.load()

In [9]:
text_splitter = CharacterTextSplitter(chunk_size=5100, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [10]:
embeddings = OpenAIEmbeddings()

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [11]:
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [12]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4",max_tokens=500)

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [14]:
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    memory=memory
)

In [15]:
query = "Are there any conditions to the closing?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/home/lillian/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The provided text does not include any information about the conditions to the closing.'

In [16]:
from IPython.display import display, Markdown

formatted_answer = f"**Answer:** \n {answer}"
display(Markdown(formatted_answer))

**Answer:** 
 The provided text does not include any information about the conditions to the closing.